# 使用 Qdrant 的 DSPy 检索器

本笔记本将指导您在 DSPy 中使用 Qdrant 作为检索器。我们将在 Qdrant 中加载数据集，并从中检索相关内容在我们的 DSPy 检索器中。

#### 设置

In [ ]:
# 安装 dspy-ai[qdrant] 包
%pip install dspy-ai[qdrant]

#### 配置常量

这个笔记本假设您在 http://localhost:6333/ 上运行了一个 Qdrant 实例。要了解有关设置 Qdrant 的更多信息，您可以参考 [快速入门指南](https://qdrant.tech/documentation/quick-start/)。

In [1]:
# 定义一个变量COLLECTION_NAME，值为"DBPEDIA-DSPY"
COLLECTION_NAME = "DBPEDIA-DSPY"
# 定义一个变量QDRANT_URL，值为"http://localhost:6333"
QDRANT_URL = "http://localhost:6333"

### 导入数据

我们将加载包含来自DBPedia的信息和使用OpenAI的`text-embedding-3-small`预先计算的嵌入的数据集[Qdrant/dbpedia-entities-openai3-text-embedding-3-small-1536-100K](https://huggingface.co/datasets/Qdrant/dbpedia-entities-openai3-text-embedding-3-small-1536-100K)！

In [ ]:
# 安装 datasets 库
%pip install datasets

In [ ]:
from datasets import load_dataset

# 导入 load_dataset 函数

# 我们将使用数据集的一个小子集
dataset = (
    load_dataset(
        "Qdrant/dbpedia-entities-openai3-text-embedding-3-small-1536-100K",
        streaming=True,
        split="train",
    )
    .take(1000)  # 仅保留前1000个样本
    .remove_columns(["openai", "combined_text"])  # 移除列 "openai" 和 "combined_text"
)

建立一个指向 http://localhost:6333/ 上的 Qdrant 实例的客户端。

In [ ]:
# 导入 QdrantClient 类
from qdrant_client import QdrantClient

# 创建 QdrantClient 实例，并传入 QDRANT_URL 参数作为 url
client = QdrantClient(url=QDRANT_URL)

我们[创建一个集合](https://qdrant.tech/documentation/concepts/collections/#create-a-collection)，该集合具有适当的维度和距离度量标准，以便将我们的数据集加载到其中。

In [4]:
from qdrant_client import models  # 导入qdrant_client模块中的models

client.create_collection(  # 调用client对象的create_collection方法
    collection_name=COLLECTION_NAME,  # 设置collection_name参数为COLLECTION_NAME变量的值
    vectors_config=models.VectorParams(  # 设置vectors_config参数为models.VectorParams对象
        size=1536,  # 设置VectorParams对象的size属性为1536
        distance=models.Distance.COSINE,  # 设置VectorParams对象的distance属性为models.Distance.COSINE
    ),
)

True

我们现在可以加载要在 Qdrant 中索引的数据集。`upload_collection` 方法接受参数来配置批处理大小和并行性。我们将使用默认设置。

In [5]:
# 从数据集中提取"text-embedding-3-small-1536-embedding"字段的数值，并将其存储在vectors列表中
vectors = [entry.pop("text-embedding-3-small-1536-embedding") for entry in dataset]

# 调用client的upload_collection方法，上传数据集到指定的COLLECTION_NAME集合中，包括vectors和完整的dataset
client.upload_collection(collection_name=COLLECTION_NAME, vectors=vectors, payload=dataset)

加载已完成。您可以浏览 http://localhost:6333/dashboard 中的条目。

#### 初始化 Qdrant 检索器和 OpenAI 向量化器

Qdrant检索器允许我们配置要使用的矢量化器。根据我们的数据集，我们将使用`OpenAIVectorizer`和`text-embedding-3-small`模型。

我们还可以在Qdrant负载中指定文档内容的字段。在我们的情况下，它是`"text"`。根据我们加载的数据集。

In [ ]:
# 导入os模块
import os

# 设置OpenAI API密钥
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

In [3]:
from dsp.modules.sentence_vectorizer import OpenAIVectorizer

# 创建一个OpenAIVectorizer对象，使用"text-embedding-3-small"模型
vectorizer = OpenAIVectorizer(model="text-embedding-3-small")

In [4]:
from dspy.retrieve.qdrant_rm import QdrantRM  # 导入 QdrantRM 类

# 创建 QdrantRM 实例
qdrant_retriever = QdrantRM(
    qdrant_client=client,  # 使用指定的 Qdrant 客户端
    qdrant_collection_name=COLLECTION_NAME,  # 指定 Qdrant 集合的名称
    vectorizer=vectorizer,  # 指定向量化器
    document_field="text",  # 指定文档字段
)

现在实例化了 `qdrant_retriever`，我们现在可以配置 `dspy` 来使用它。

In [5]:
# 导入dspy模块
import dspy

# 配置dspy模块的设置，使用qdrant_retriever作为参数
dspy.settings.configure(rm=qdrant_retriever)

### 尝试检索器

我们可以使用 `dspy.Retrieve` 类来查询我们的检索器。类似于在 DSPy RAG 管道中的操作方式。

In [6]:
# 创建一个Retrieve对象
retrieve = dspy.Retrieve()

# 使用Retrieve对象检索文本"Some computer programs."
retrieve("Some computer programs.")

Prediction(
    passages=['CounterSpy is a proprietary spyware removal program for Microsoft Windows software developed by Sunbelt Software.', 'In computing, the diff utility is a data comparison tool that calculates and displays the differences between two files. Unlike edit distance notions used for other purposes, diff is line-oriented rather than character-oriented, but it is like Levenshtein distance in that it tries to determine the smallest set of deletions and insertions to create one file from the other.', "AudioDesk is an audio workstation application by Mark of the Unicorn (MOTU) for the Mac OS. It is a multi-track recording, editing, and mixing application, with both offline file-based processing and realtime effects. It is a more basic version of MOTU's Digital Performer  DAW software. Much of the graphical user interface (GUI) and its operation are similar to Digital Performer, although it lacks some of Digital Performer's features."]
)

我们能够成功地从我们的Qdrant集合中检索与查询相关的结果。